**Import**

In [1]:
import os
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from config import *

In [11]:
model = load_model('vgg_model_150X150.h5')

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 15,764,037
Trainable params: 1,049,349
Non-trainable params: 14,714,688
_________________________________________________________________


In [13]:
train_df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'))
train_df['label'] = train_df['label'].astype(str)

In [14]:
train_df.sample(5)

,image_id,label
8855,2567910282.jpg,1
5647,2003348266.jpg,4
6129,2094252123.jpg,3
9284,2644492010.jpg,2
21300,979717744.jpg,3


In [15]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [23]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=os.path.join(BASE_DIR, 'train_images'),
    x_col='image_id',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
)

Found 21397 validated image filenames belonging to 5 classes.


In [21]:
earlystop = EarlyStopping(monitor="val_loss", patience=3, verbose=1, mode='auto')
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

In [25]:
history = model.fit(train_generator,
                    epochs = 1,
                    callbacks=[earlystop])

669/669 [==============================] - 2369s 4s/step - loss: 1.0085 - accuracy: 0.6262


In [26]:
# save model and architecture to single file
model.save("vgg_model_150X150.h5")
print("Saved model to disk")

Saved model to disk


In [28]:
filenames = []
labels = []

In [29]:
for f in os.listdir(os.path.join(BASE_DIR, 'test_images')):
    filenames.append(f)
    labels.append('0')

test_df = pd.DataFrame({
    'image_id': filenames,
    'label': labels,
})

In [16]:
col_dict = dict(zip(np.arange(10), y.columns))
col_dict

{0: 'keyword_animal',
 1: 'keyword_human',
 2: 'keyword_mountain',
 3: 'keyword_nature',
 4: 'keyword_outdoors',
 5: 'keyword_person',
 6: 'keyword_plant',
 7: 'keyword_sea',
 8: 'keyword_tree',
 9: 'keyword_water'}